In [9]:
import wikipediaapi
import time
import re
import pickle
from ipywidgets import IntText, Text, Box
from IPython.display import display

In [57]:
class Harvester:
    def __init__(self, rsize):
        self.rsize = rsize
        self.res = set()
        self.resdict = []
        self.lookup = set()

    def _harvester(self, member, *, wiki_wiki, word, percent, text, gender, check=False):
        """
        member - wiki page title
        wiki_wiki - language for wiki
        res - final set of entities
        check - regular expression with a list of undesired subcategories like 'inhabitants' inside a city category or False if you don't want to check anything
        percent - technical variable for rendering the progress
        text - technical variable for rendering current category
        """
        if len(self.res) >= self.rsize:
            return 
        time.sleep(1)
        cat = wiki_wiki.page(member.title)
        try:
            for p in cat.categorymembers.values():
                if check:
                    if not (re.search(check, p.title) or p.title in self.lookup):  # we don't want to go inside an unwanted subcategory
                        text.value = p.title
                        self.lookup.add(p.title)
                        self._harvester(p, wiki_wiki=wiki_wiki, word=word, check=check, percent=percent,gender=gender,  text=text)
                else:
                    self._harvester(p, wiki_wiki=wiki_wiki, word=word, check=check, percent=percent, gender=gender, text=text)
        except KeyError:
            if re.search(r'(?i)\b(список|спіс|spis|lista|kategoria|катэгорыя|категор[іи]я|відносини)\b', member.title):  # universal check for unwanted pages
                return    
            try:
                for key in member.categories.keys():
                    if re.search(word, key):
                        langlinks = member.langlinks
                        middle = {'be': member.title}
                        for lr in ['bg', 'ca', 'cs', 'en', 'es', 'pl', 'ru', 'sl', 'tr', 'uk']:
                            v = langlinks.get(lr)
                            if not v: 
                                text.value = f'{member.title} has no {lr}'
                                return
                            elif v:
                                middle[lr] = langlinks.get(lr).title
                        for k in member.categories.keys():
                            if re.search(gender, k):
                                middle['gender'] = 'f'
                                break
                        else:
                            middle['gender'] = 'm'
                        if member.title not in self.res:
                            self.res.add(member.title)
                            self.resdict.append(middle)
                        percent.value = len(self.res)
                        return 
                    else:
                        continue  
            except:
                return 

    def harvest(self, *, lang, category, word, gender, check=False):
        '''Wrapper function for harverster with check'''
        wiki_wiki = wikipediaapi.Wikipedia(lang)
        cat = wiki_wiki.page(category)

        percent = IntText(value=0, disabled=True)
        text = Text(value=None, disabled=True)
        box = Box(children=[percent, text])
        display(box)
        
        self._harvester(cat, wiki_wiki=wiki_wiki, word=word, check=check, percent=percent, gender=gender, text=text)

    def writeres(self, lang, type):
        pickle.dump(self.resdict, open(f'names/{lang}_{type}_{len(self.res)}', 'wb'))

    def showres(self, to_show):
        i = 0
        for key in self.resdict:
            if i >= to_show:
                break
            i += 1
            for k, v in key.items():
                print(k, v, sep=': ', end='\t')
                print()

    def results(self):
        return self.res

    def renew(self):
        self.res = set()

    def help(self):
        print('Use harvest function. Variables: lang=language, category=head category to start, word=check word for final entity addition, check(optional)=regular expression for filtering subcategories')

**PER entities**

In [60]:
worker = Harvester(3000)
worker.renew() 
worker.harvest(lang='be', category='Катэгорыя:Нарадзіліся ў XIX стагоддзі', word='[Нн]арадзіліся', gender=re.compile(r'(?i)жанчыны|жонкі|\wенкі\b|\wісткі\b|\wніцы\b|\wіні\b|прынцэсы|актрысы|куртызанкі|прастытуткі|натуршчыцы|манашкі|\wчыцы\b'))
worker.writeres('be', 'per')
worker.showres(10)

Box(children=(IntText(value=0, disabled=True), Text(value='', disabled=True)))

be: Джон Браўн	
bg: Джон Браун	
ca: John Brown (abolicionista)	
cs: John Brown	
en: John Brown (abolitionist)	
es: John Brown (abolicionista)	
pl: John Brown	
ru: Браун, Джон (аболиционист)	
sl: John Brown (abolicionist)	
tr: John Brown (lider)	
uk: Джон Браун	
gender: m	
be: Фрыдрых Вёлер	
bg: Фридрих Вьолер	
ca: Friedrich Wöhler	
cs: Friedrich Wöhler	
en: Friedrich Wöhler	
es: Friedrich Wöhler	
pl: Friedrich Wöhler	
ru: Вёлер, Фридрих	
sl: Friedrich Wöhler	
tr: Friedrich Wöhler	
uk: Фрідріх Велер	
gender: m	
be: Гельмут Карл Бернхард фон Мольтке (старэйшы)	
bg: Хелмут фон Молтке Старши	
ca: Helmuth von Moltke	
cs: Helmuth von Moltke starší	
en: Helmuth von Moltke the Elder	
es: Helmuth Karl Bernhard von Moltke	
pl: Helmuth Karl Bernhard von Moltke	
ru: Мольтке, Хельмут Карл Бернхард фон	
sl: Helmuth Karl Bernhard von Moltke	
tr: Helmuth Karl Bernhard von Moltke	
uk: Гельмут Карл Бернхард фон Мольтке	
gender: m	
be: Францэ Прашэрн	
bg: Франце Прешерн	
ca: France Prešeren	
cs: France P

In [ ]:
worker = Harvester(10)
worker.renew()
worker.harvest(lang='pl', category='Kategoria:Wokalistki popowe', word='[uU]rodzeni', gender=re.compile(r'(?i)bokserki|kobiety|\wstki\b|iczki\b|[aeo]rki\b|damy|[aes][nt]ki\b|zakonnice|inie\b|żony|cezarzowe|królowe'))
worker.writeres('pl', 'per')
worker.showres(10)

In [ ]:
worker = Harvester(5000)

# ru

worker.harvest(lang='ru', category='Категория:Родившиеся в XX веке', word='родившиеся')
worker.writeres('ru', 'per')

# be

worker.renew()
worker.harvest(lang='be', category='Катэгорыя:Нарадзіліся ў XX стагоддзі', word='нарадзіліся')
worker.writeres('be', 'per')

# uk

worker.renew()
worker.harvest(lang='uk', category='Категорія:Народились у 20 столітті', word='народил')
worker.writeres('uk', 'per')

# pl

worker.renew()
worker.harvest(lang='pl', category='Kategoria:Urodzeni w XX wieku', word='urodzeni')
worker.writeres('pl', 'per')

**LOC entities**

In [4]:
worker = Harvester(3000)

In [ ]:
worker.harvest(lang='be', category='Катэгорыя:Дзяржавы паводле алфавіта', word='дзяржавы')
cities = re.compile(r'Архітэктура|Гісторыя|Постаці|Адукацыя|Арганізацыі|Вуліцы|Культура|Паркі|Спорт|Транспарт')
worker.harvest(lang='be', category='Катэгорыя:Сталіцы', word='гарады', check=cities)
worker.writeres('be', 'loc')

In [19]:
countries = re.compile(r'(?i)Колишні|Географі|Футбол|Кавалери|монархі|Геологі|Іспанці|Святий Престол|Економі|Історія|Еміратів|Округи|Нагірний Карабах|Дипломати|Посли|орден|Біота|копалини|Культ|Померли|Уродженці|Підприємства|Політи[кч]|Транспорт|Уряди|Незавершені статті|Збройні сили|Зображення|Населення|Наук|Персоналії|Природа|Списки|Спорт|Суспільство|Шаблони|Фауна|Портал|Столиці|Архітектура|Історичні|діаспора|Військові|Категорії|Окуповані|Право|Вікіпроєкт|Інтернет|Французи|Чехи|Шведи|Японці|Фільми|поділ|Гідрографія|Рельєф|Штати|Державна влада|Відносини|Техніка|символи|Міс[тц]|Села|Учасники|Фортеці|мова|Регіони|райони')
worker.harvest(lang='uk', category='Категорія:Країни за континентом', word='країни', check=countries)
print('cities')
cities = re.compile(r'Архітектура|Географія|Історія|Кавалери|Культ|Освіта|Святий Престол|Персоналії|Підприємства|Спорт|Транспорт|Померли|Уродженці|орден|Випускники|Дипломати|Шаблони|Списки|Вулиці|Зображення|Організації|Парки|Площі|Райони|Туризм|Цвинтарі|Портал|Незавершені статті|Військові формування|Економіка|Медицина|Населення|Наук|Об\'єкти|Політи[кч]|Природа|Суди|поділ|ЗМІ|діаспора|Релігія|Романи|Квартали|місц|заклади|академи|університ|споруди')
worker.harvest(lang='uk', category='Категорія:Столиці', word='столиці', check=cities)
worker.writeres('uk', 'loc')

Box(children=(IntText(value=0, disabled=True), Text(value='', disabled=True)))

cities


Box(children=(IntText(value=0, disabled=True), Text(value='', disabled=True)))

In [ ]:
worker.renew()
countries = re.compile(r'historyczne|Kategorie|Ludzie|Architektura|Geografia|Gospodarka|Historia|Kultura|Polityka|Sport|Stosunki|Symbole|Etnografia|[Pp]odział|Geologia|Meksykanie|odznaczenia|Nauka|Turystyka|Wojsko|ńskie|Zagadnienia|Edukacja|[pP]rzyrod|Prawo|Siły Zbrojne|Społeczeństwo|Superkomputery|Wydarzenia|Sztuka|Stolice|Flagi|Język|Biblioteka|Konklawe|Media|Poch[oó]w|Utwory|Szablony|Herby|Klasztory|[oO]rganizacje|Oświata|Parki|Policja|Samorząd|Armie|Filmy|Konflikt|Politycy|Prasa|Rada|Adwokatura|Krajoznawstwo|Konserwatyzm|Ochrona|Stereotypy|Urbanistyka|Więziennictwo|Miejsca')
worker.harvest(lang='pl', category='Kategoria:Państwa według kontynentów', word='państwa', check=countries)
cities = re.compile(r'Ludzie|Igrzyska|Obiekty|Religia')
worker.harvest(lang='pl', category='Kategoria:Stolice', word='miasta', check=cities)
worker.writeres('pl', 'loc')

In [ ]:
worker.renew()
worker.harvest(lang='ru', category='Категория:Государства по алфавиту', word='государств')
countries = re.compile(r'(?i)География|уезды|Здания|сооружения|История|Культура|Организации|Персоналии|Районы|Спорт|университет|Небоскрёбы|Компании|деление|Транспорт|заведения|Образование|транспорт')
worker.harvest(lang='ru', category='Категория:Города-миллионеры по странам', word='города')
worker.writeres('ru', 'loc')

ORG entities

In [8]:
worker = Harvester(1000)

In [4]:
#be

worker.renew()
# checker = re.compile(r'')
worker.harvest(lang='be', category='Катэгорыя:Арганізацыі паводле алфавіта', word=re.compile('заснаваныя|арганізацыі|калектывы'))
worker.writeres('be', 'org')

Box(children=(IntText(value=0, disabled=True), Text(value='', disabled=True)))

In [5]:
#uk

worker.renew()
checker = re.compile(r'(?i)Члени|Викладачі|Випускники|Учні|Науковці|Ректори|Військові|НАТО|Люди|фільми|програми|\w+істи|Єпископи|Вікаріати|Єпархії|Монастирі|Нагороди|Священники|храми|Персоналії')
worker.harvest(lang='uk', category='Категорія:Організації, засновані у XX столітті', word='засновані', check=checker)
worker.harvest(lang='uk', category='Категорія:Організації, засновані у XXI столітті', word='засновані', check=checker)
worker.writeres('uk', 'org')

Box(children=(IntText(value=0, disabled=True), Text(value='', disabled=True)))

Box(children=(IntText(value=0, disabled=True), Text(value='', disabled=True)))

In [ ]:
#ru

worker.renew()
checker = re.compile(r'(?i)монастыри|учебные|библиотеки|театры')
worker.harvest(lang='ru', category='Категория:Организации, основанные в XX веке', word='основанные', check=checker)
# worker.harvest(lang='uk', category='Категорія:Організації, засновані у XXI столітті', word='організації', check=checker)
worker.writeres('ru', 'org')

In [8]:
#pl

worker.renew()
checker = re.compile(r'(?i)Kustosz|Ludzie|Lustracja|Periodyki|Pracownicy|Członk|ratownicze|muzea|Działacze|Żołnierze|sportowe|teatry|Nieistniejące|Szkoły|Legiony|Politycy|Bitwy|Odznaczeni|Ofiary|Okręgi|Prasa|Oddział|według|absolwenci|Wykładowcy|Rektorzy|bibliotek')
worker.harvest(lang='pl', category='Kategoria:Organizacje według państw', word='(?i)organizacje|partie|przedsiębiorstwa|stowarzyszenia|fundacje|Instytuty', check=checker)
# checker = re.compile(r'(?i)Kustosz|Ludzie|Lustracja|Periodyki|Pracownicy|Członkowie|Działacze|Politycy|Nieistniejące|Żołnierze|Bitwy|Ofiary|Okręgi|Prasa|Oddział')
# worker.harvest(lang='pl', category='Kategoria:Partie i ugrupowania według państw', word='(?i)partie', check=checker)
worker.writeres('pl', 'org')

Box(children=(IntText(value=0, disabled=True), Text(value='', disabled=True)))

In [10]:
worker.writeres('ru', 'org')

Parallel entities

In [2]:
def get_parallels(targets, lrs, ld):
    wiki_wiki = wikipediaapi.Wikipedia(ld)
    res = []
    for target in targets:
        middle = {ld: target}
        page = wiki_wiki.page(target)
        langlinks = page.langlinks
        for lr in lrs:
            v = langlinks.get(lr)
            if v:
                middle[lr] = langlinks.get(lr).title
            else:
                middle[lr] = None
        res.append(middle)
    return res

In [3]:
br = pickle.load(open(r'names\be_loc_575', 'rb'))

In [4]:
langs = get_parallels(br, ['ru', 'pl', 'uk'], 'be')